# Database Ingestion Scrip - California wild-fires (2013-2019)

This script contains:
1. Relevent libraries import, and create_connection, create_table & execute_sql_statement functions.
2. DB - normalized.db file creation
3. Pandas DataFrames to deal with missing values and dropping irrelevant columns for this project
4. Creation of "County_Details", "Location_Details", and "Incident_Details" tables using respective functions
5. Running all functions to complete the ingestion script for further analysis 

In [1]:
### Utility Functions
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error
import os
import csv

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [2]:
normalized_database_filename = 'normalized.db'
# Delete old database file
conn_norm = create_connection(normalized_database_filename, delete_db=True)

In [3]:
df_cal = pd.read_csv('California_Fire_Incidents.csv')

In [4]:
# Updating the missing values of dataframe - 'df_cal' with - 'Fatalities' and 'Injuries' as zero and 'AcresBurned' as median 
# of the 'AcresBurned' and storing in 'df_cal_updated'
df_cal_updated = df_cal.fillna({'Fatalities' : 0, 'Injuries': 0, 'AcresBurned': df_cal['AcresBurned'].median(), 'Extinguished': 0})
# dropping - AirTankers, ConditionStatement, ControlStatement, CrewsInvolved, Dozers, Engines, Extinguished, FuelType,
# Helicopters, PersonnelInvolved, SearchDescription, SearchKeywords, StructuresDamaged, StructuresDestroyed,
# StructuresEvacuated, StructuresThreatened, WaterTenders
df_cal_updated = df_cal_updated.dropna(axis = 1, how = 'any')
#df_cal_updated[['Active','Name','Started','Status']]
df_cal_updated['fire_duration'] = (pd.to_datetime(df_cal_updated.Extinguished) - pd.to_datetime(df_cal_updated.Started)).astype('timedelta64[h]')/24 # To convert in days
df_cal_updated['fire_duration'] = pd.to_numeric(df_cal_updated.fire_duration,errors = 'coerce')

In [5]:
updated_csv = 'California_Fire_Incidents_Cleaned.csv'
if os.path.exists(updated_csv):
    os.remove(updated_csv)
df_cal_updated.to_csv(updated_csv)

# Normalized database schema

    1. County_Details table has two columns:
        [CountyID] column is the primary key
        [County] column stores the name of the county

    2. Location_Details table has five columns:
        [LocationID] column is the primary key column
        [Latitude] column stores latitude
        [Longitude] column stores longitude
        [Location] column stores the location details
        [CountyID] foreign key to County_Details table

    3. Incident_Details table has seven columns:
        [IncidentID] primary key column 
        [LocationID] foreign key to Location_Details table
        [Year] column stores the archive year of the fire incident
        [AcresBurned] column stores the acres of area burnt in each fire incident
        [StartDate] column stores the timestamp of the start date and time of the fire
        [Duration] column stores the duration of each fire incident in days
        [Fatalities] column stores the number of human lives succumbed at each fire incident

In [6]:
def County_Details(data_filename):
    '''This function creates a normalized table to store all counties affected in the california fires.
    The table generater has CountyIDs generated as the primary key for the table.
    Note: the table created is using SQLite module as database table.'''
    
    create_table_sql = """create table County_Details(CountyID Integer not null primary key,
                                                        County Text not null);"""
    
    create_table(conn_norm, create_table_sql, drop_table_name='County_Details')
    
    # Initializing necessary lists
    header = []
    county_name = []
    # Iterating through columns of the csv file 'California_Fire_Incidents_Cleaned_2.csv' to extract necessary data
    with open(data_filename) as file:
        line_read = csv.reader(file)
        lines = list(line_read)
        for i in range(len(lines)):
            if header == []:
                header.append(lines[i])
            else:
                if (lines[i][7],) in county_name:
                    continue
                else:
                    county_name.append((lines[i][7],))
                    
    # updating values in "County_Details" table:
    with conn_norm:
        cur = conn_norm.cursor()
    
        cur.executemany("""INSERT INTO County_Details(County)
                    VALUES(?)""", county_name)
    
    conn_norm.commit()
    
#Function call:
County_Details(updated_csv)

In [7]:
def County_dictionary(data_filename):
    '''This function creates the dictionary with key as the counties and value as the CountyID.
    The purpose of this dictionary is to inject foreign keys in the "Location_Details" table.'''
    
    sql = """select CountyID, County from County_Details"""
    locations = execute_sql_statement(sql, conn_norm)
    County_dict = {}
    for ele in locations:
        County_dict[ele[1]] = ele[0]
        
    return County_dict
County_dict = County_dictionary(updated_csv)

In [8]:
def Location_Details(data_filename):
    '''This function is meant to create "Location_Details" table that contains the unique location details.
    Here the "LocationID" is primary key and the "CountyID" is foreign key from the "County_Details" table.
    The Latitude, Longitude & Location is extracted from the 'California_Fire_Incidents_Cleaned_2.csv' file.
    Note: the table created is using SQLite module as database table.'''
    
    create_table_sql = """create table Location_Details(LocationID Integer not null primary key,
                                                        Latitude float not null,
                                                        Longitude float not null,
                                                        Location text not null,
                                                        CountyID text not null,
                                                        foreign key(CountyID) REFERENCES County_Details(CountyID));"""
    
    create_table(conn_norm, create_table_sql, drop_table_name='Location_Details')
    
    # Initializing necessary lists
    header = []
    incident_loc = []
    lat_long = []
    # Iterating through columns of the csv file 'California_Fire_Incidents_Cleaned_2.csv' to extract necessary data
    with open(data_filename) as file:
        line_read = csv.reader(file)
        lines = list(line_read)
        for i in range(len(lines)):
            if header == []:
                header.append(lines[i])
            else:
                if (lines[i][14],lines[i][16]) in lat_long:
                    continue
                else:
                    incident_loc.append((float(lines[i][14]),float(lines[i][16]),lines[i][15],County_dict[lines[i][7]],))
                    lat_long.append((lines[i][14],lines[i][16],))
                    
    # updating values in "Location_Details" table:
    with conn_norm:
        cur = conn_norm.cursor()
    
        cur.executemany("""INSERT INTO Location_Details(Latitude, Longitude, Location, CountyID)
                    VALUES(?,?,?,?)""", incident_loc)
    
    conn_norm.commit()
    
#Function call:
Location_Details(updated_csv)

In [9]:
def location_dictionary(data_filename):
    '''This function creates the dictionary with key as the (Latitude, Longitude) and value as the LocationID.
    The purpose of this dictionary is to inject foreign keys in the "Incident_Details" table.'''
    
    sql = """select LocationID, Latitude, Longitude from Location_Details"""
    locations = execute_sql_statement(sql, conn_norm)
    location_dict = {}
    for ele in locations:
        location_dict[ele[1],ele[2]] = ele[0]
        
    return location_dict

location_dict = location_dictionary(updated_csv)

In [10]:
def Incident_Details(data_filename):
    '''This is a master table that contains the list of all incidents from 2013 to 2019.
    The IncidentID the primary key for this table.
    The LocationID is the foreign key from the "Location_Details" table.
    The Year, AcresBurned, StartDate & Duration is extracted from the 'California_Fire_Incidents_Cleaned_2.csv' file.
    Note: the table created is using SQLite module as database table.'''
    
    create_table_sql = """create table Incident_Details(IncidentID Integer not null primary key,
                                                        LocationID Integer not null,
                                                        Year Integer not null,
                                                        AcresBurned float not null,
                                                        StartDate text not null,
                                                        Duration float not null,
                                                        Fatalities integer not null,
                                                        foreign key(LocationID) REFERENCES Location_Details(LocationID));"""

    create_table(conn_norm, create_table_sql, drop_table_name='Incident_Details')
    
    # Initializing necessary lists
    header = []
    incident = []
    location = []
    # Iterating through columns of the csv file 'California_Fire_Incidents_Cleaned_2.csv' to extract necessary data
    with open(data_filename) as file:
        line_read = csv.reader(file)
        lines = list(line_read)
        for i in range(len(lines)):
            if header == []:
                header.append(lines[i])
            else:
                lat = float(lines[i][14])
                long = float(lines[i][16])
                if float(lines[i][24]) < 0:
                    value = 0
                else:
                    value = float(lines[i][24])
                if location_dict[(lat,long)] not in location:
                    location.append(location_dict[(lat,long)])
                    incident.append((location_dict[(lat,long)], int(lines[i][4]),float(lines[i][1]),lines[i][23],value,lines[i][10],))
    
    # updating values in "Incident_Details" table:
    with conn_norm:
        cur = conn_norm.cursor()
    
        cur.executemany("""INSERT INTO Incident_Details(LocationID, Year, AcresBurned, StartDate, Duration, Fatalities)
                    VALUES(?,?,?,?,?,?)""", incident)
    
    conn_norm.commit()
    
#Function call:
Incident_Details(updated_csv)

In [11]:
df1 = pd.read_sql_query("""SELECT * FROM County_Details""", conn_norm)
df2 = pd.read_sql_query("""SELECT * FROM Location_Details""", conn_norm)
df3 = pd.read_sql_query("""SELECT * FROM Incident_Details""", conn_norm)

In [12]:
display(df1)

,CountyID,County
0,1,Tuolumne
1,2,Los Angeles
2,3,Riverside
3,4,Placer
4,5,Ventura
5,6,Fresno
6,7,Siskiyou
7,8,Humboldt
8,9,Tehama
9,10,Shasta


In [13]:
display(df2)

,LocationID,Latitude,Longitude,Location,CountyID
0,1,37.857000,-120.086000,3 miles east of Groveland along Hwy 120,1
1,2,34.585595,-118.423176,Angeles National Forest,2
2,3,33.709500,-116.728850,Hwy 243 & Hwy 74 near Mountain Center,3
3,4,39.120000,-120.650000,"Deadwood Ridge, northeast of Foresthill",4
4,5,0.000000,0.000000,Southbound Highway 101 at Camarillo Springs Ro...,5
...,...,...,...,...,...
1444,1445,33.827979,-117.499619,"Eagle Canyon Rd. and Cajalco Rd., southwest of...",3
1445,1446,39.409722,-121.000556,"Off of Long Point Road and Old Mill Road, Sou...",42
1446,1447,38.734634,-121.729691,"County Road 102 and County Road 17, North of W...",33
1447,1448,33.351145,-117.403719,"Near Basilone Road and Las Pulgas Road, near C...",11


In [14]:
display(df3)

,IncidentID,LocationID,Year,AcresBurned,StartDate,Duration,Fatalities
0,1,1,2013,257314.0,2013-09-06T18:30:00Z,20.125000,0
1,2,2,2013,30274.0,2013-06-08T18:30:00Z,9.125000,0
2,3,3,2013,27531.0,2013-07-30T18:00:00Z,15.166667,0
3,4,4,2013,27440.0,2013-08-30T08:00:00Z,19.625000,0
4,5,5,2013,24251.0,2013-05-11T06:30:00Z,8.958333,0
...,...,...,...,...,...,...,...
1444,1445,1445,2019,9.0,2019-10-10T18:11:59.733Z,0.250000,0
1445,1446,1446,2019,2.0,2019-06-30T15:52:01.023Z,0.083333,0
1446,1447,1447,2019,100.0,2019-12-03T16:35:20.93Z,0.000000,0
1447,1448,1448,2019,100.0,2019-11-21T12:21:28.58Z,0.000000,0
